In [4]:
# !pip install langchain
# !pip install pinecone-client
# !pip install pinecone
# !pip install sentence_transformers
# !pip install huggingface_hub==0.25.2
# !pip install "pinecone[grpc]"

#!pip3 install pinecone==5.0.0




# Import Directories

In [ ]:
import pinecone
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from pinecone import Pinecone, ServerlessSpec
from langchain.vectorstores import Pinecone
from sentence_transformers import SentenceTransformer

from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import time



c:\Users\admin\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [ ]:
#PINECONE_API_KEY = "pcsk_4hZwV6_Qk4yj6FHJGPEqkii7FQFeZ3junJuKnx4Qmnxw4MCk8GQTUh7HM19knvJe41hVef"
#PINECONE_API_ENV = "us-east1-aws"

In [ ]:
# Initialize a Pinecone client with your API key
pc = Pinecone(api_key="pcsk_4hZwV6_Qk4yj6FHJGPEqkii7FQFeZ3junJuKnx4Qmnxw4MCk8GQTUh7HM19knvJe41hVef")



# Create an index with the correct dimension (384)
pinecone.create_index(index_name, dimension=384, metric="cosine")

# Connect to the index
index = pinecone.Index(index_name)

# Use HuggingFaceEmbeddings to create embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Creating Embeddings for each of the text chunks and storing
text_chunks = ["your text chunk 1", "your text chunk 2", ...]  # Replace with your actual text chunks
docsearch = Pinecone.from_texts(
    texts=[t for t in text_chunks],
    embeddings=embeddings,
    index_name=index_name
)

# Example query
query_text = "your query text"
query_embeddings = embeddings.encode([query_text])
result = index.query(queries=query_embeddings, top_k=5)
print(result)

# Optional: Delete the index
pinecone.delete_index(index_name)

In [37]:
# Extract data from the pdf
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)

    documents = loader.load()
    
    return documents

In [38]:
extracted_data = load_pdf("data/")

In [39]:
#extracted_data

In [ ]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 50)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks
embeddings = download_hugging_face_embeddings()




In [41]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 20990


In [42]:
#text_chunks

In [43]:
# from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
print("Model loaded successfully")


Model loaded successfully


In [44]:
embeddings = download_hugging_face_embeddings()
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [45]:
#embeddings

In [46]:
query_result = embeddings.embed_query("Hello World")
print("length", len(query_result))

length 384


In [47]:
#query_result

In [ ]:
#vdo
#Initializing the Pinecone
#pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)
pinecone.init(api_key=PINECONE_API_KEY)


index_name = "medical-chatbot"

#Creating Embeddings for Each of the Text Chunks and storing
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name)


In [ ]:
#If we already have an index we can load it like this
docsearch = Pinecone.from_existing_index(index_name, embeddings)

query = "What are Allergies"

docs = docsearch.similarity_search(query, k=3)

print("Result", docs)

In [ ]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [ ]:
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs = {"prompt": PROMPT}

In [ ]:
llm = CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={'max_new_tokens':512,
                            'temperature':0.8})

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)

In [ ]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

# Below code is just observe not need to execute

In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
pc = Pinecone(api_key="pcsk_6K37fx_4ojcVFjUNuemDqFEp8bs3oGYu2gX8XeL6ACbjGqtFwciSWqrFu7hXaFsiHqWqQD")
index = pc.Index("medical-chatbot")

#Creating Embeddings for Each of the Text Chunks and storing
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index)

In [56]:
from google.api import annotations_pb2
from google.protobuf import json_format

print("Modules imported successfully")


Modules imported successfully


In [ ]:
import pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone

# # Set your Pinecone API key
PINECONE_API_KEY = "pcsk_6K37fx_4ojcVFjUNuemDqFEp8bs3oGYu2gX8XeL6ACbjGqtFwciSWqrFu7hXaFsiHqWqQD"

# Initialize Pinecone
pinecone.init(api_key=PINECONE_API_KEY)

# Define your index name
index_name = "medical-chatbot"

# Connect to the existing index
index = pinecone.Index(index_name)

# Use HuggingFaceEmbeddings to create embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")




In [ ]:


# Creating Embeddings for each of the text chunks and storing
#text_chunks = ["your text chunk 1", "your text chunk 2", ...]  # Replace with your actual text chunks
docsearch = Pinecone.from_texts(
    texts=[t for t in text_chunks],
    embeddings=embeddings,
    index_name=index_name
)

# Example query
query_text = "your query text"
query_embeddings = embeddings.encode([query_text])
result = index.query(queries=query_embeddings, top_k=5)
print(result)

# Optional: Delete the index
# pinecone.delete_index(index_name)

# Serverless index
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key="YOUR_API_KEY")

pc.create_index(
    name="example-index1",
    dimension=1536,
    metric="cosine",
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

In [ ]:
import pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Set your Pinecone API key
PINECONE_API_KEY = "pcsk_6K37fx_4ojcVFjUNuemDqFEp8bs3oGYu2gX8XeL6ACbjGqtFwciSWqrFu7hXaFsiHqWqQD"

# Initialize Pinecone Client
client = pinecone.Client(api_key=PINECONE_API_KEY)

# Define your index name
index_name = "medical-chatbot"

# Check if the index already exists
if index_name not in client.list_indexes():
    client.create_index(
        name=index_name,
        dimension=384,
        metric="cosine",
        pod_type="p1",  # Specify pod type for serverless index
        replicas=1,     # Number of replicas
        cloud="aws",
        region="us-east-1"
    )

# Connect to the existing index
index = client.Index(index_name)



# Function to create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
    return text_splitter.split_documents(extracted_data)

# Function to download embedding model
def download_hugging_face_embeddings():
    return HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Use the provided functions
extracted_data = [{"page_content": "Your extracted text content 1."}, {"page_content": "Your extracted text content 2."}]
text_chunks = text_split(extracted_data)
embeddings = download_hugging_face_embeddings()

# Creating Embeddings for each of the text chunks and storing them in Pinecone
docsearch = Pinecone.from_texts(
    texts=[t["page_content"] for t in text_chunks],
    embeddings=embeddings,
    index_name=index_name
)

# Example query
query_text = "your query text"
query_embeddings = embeddings.encode([query_text])
result = index.query(queries=query_embeddings, top_k=5)
print(result)

# Optional: Delete the index
# client.delete_index(index_name)


In [ ]:
import pinecone
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Set your Pinecone API key
PINECONE_API_KEY = "pcsk_6K37fx_4ojcVFjUNuemDqFEp8bs3oGYu2gX8XeL6ACbjGqtFwciSWqrFu7hXaFsiHqWqQD"

# Initialize Pinecone
pinecone.init(api_key=PINECONE_API_KEY, environment="us-west1-gcp")

# Define your index name
index_name = "medical-chatbot"

# Check if the index already exists
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name,
        dimension=384,
        metric="cosine",
        pod_type="p1"
    )

# Connect to the existing index
index = pinecone.Index(index_name)

# Function to create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

# Function to download embedding model
def download_hugging_face_embeddings():
    return HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Use the provided functions
extracted_data = [{"page_content": "Your extracted text content 1."}, {"page_content": "Your extracted text content 2."}]
text_chunks = text_split(extracted_data)
embeddings = download_hugging_face_embeddings()

# Creating Embeddings for each of the text chunks and storing them in Pinecone
docsearch = Pinecone.from_texts(
    texts=[t["page_content"] for t in text_chunks],
    embeddings=embeddings,
    index_name=index_name
)

# Example query
query_text = "your query text"
query_embeddings = embeddings.encode([query_text])
result = index.query(queries=query_embeddings, top_k=5)
print(result)

# Optional: Delete the index
# pinecone.delete_index(index_name)
